In [ ]:
!pip install transformers
!pip install transformers[torch]

!pip install transformers[torch] accelerate -U datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install torch transformers

In [ ]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from google.colab import drive

In [ ]:
# 구글 드라이브 마운트
drive.mount('/content/drive')



In [ ]:
# 데이터 경로 설정 (구글 드라이브 내의 경로)
data_path = '/content/drive/MyDrive/01.데이터/1.Training/라벨링데이터'

In [ ]:
import json

# JSON 파일 경로
input_file_path = 'all_texts.json'

# JSON 파일을 읽어서 all_texts 리스트에 저장
with open(input_file_path, 'r', encoding='utf-8') as f:
    all_texts = json.load(f)

# 읽은 데이터 확인
print(f'Read {len(all_texts)} dialogues from {input_file_path}')

In [ ]:



# 텍스트 데이터를 결합하여 GPT 모델 입력용 텍스트 생성
dataset_text = "\n".join(all_texts)

# 토크나이저 설정
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  #는 Hugging Face의 Transformers 라이브러리에서 제공하는 코드로, GPT-2 모델에 맞는 토크나이저를 로드,from_pretrained 메서드 사용: 이 메서드는 지정된 이름('gpt2')에 해당하는 사전 훈련된 토크나이저를 다운로드하고 로드합니다.
tokenizer.pad_token = tokenizer.eos_token



In [ ]:
# 토큰화된 데이터 생성
inputs = tokenizer(dataset_text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs):
        self.input_ids = inputs['input_ids']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'labels': self.input_ids[idx]
        }

dataset = CustomDataset(inputs)

In [ ]:
# 모델 설정
model = GPT2LMHeadModel.from_pretrained('gpt2')

# 데이터 콜레이터 설정 #특히, 배치 내에서 시퀀스의 길이를 맞추고, 모델이 입력으로 사용할 수 있는 형태로 데이터를 준비하는 데 중요한 역할
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
print(data_collator)

DataCollatorForLanguageModeling(tokenizer=GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')


In [ ]:


# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    prediction_loss_only=True
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# 모델 학습
trainer.train()

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Step,Training Loss


TrainOutput(global_step=3, training_loss=2.472482363382975, metrics={'train_runtime': 13.7196, 'train_samples_per_second': 0.219, 'train_steps_per_second': 0.219, 'total_flos': 783876096000.0, 'train_loss': 2.472482363382975, 'epoch': 3.0})

In [ ]:
# GPU 또는 CPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 챗봇 응답 생성 함수
def generate_response(model, tokenizer, input_text, max_length=50):
    model.eval()
    inputs = tokenizer.encode(input_text, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response



In [ ]:
tokenizer.eos_token_id

50256

In [ ]:
# 예시 입력 텍스트
input_text = "머먹었어?"

# 모델을 사용하여 응답 생성
response = generate_response(model, tokenizer, input_text)
print(f"Response: {response}")

Response: 머먹었어?

아리리었어?

아리리었어?

